# Hex to RGB

Can I train a neural network to convert hex color codes to RGB?  

## Generate some sample data

In [53]:
def hex_to_rgb(hex):
  hex = hex.replace('#', '')
  rgb = []
  for i in (0, 2, 4):
    decimal = int(hex[i:i+2], 16)
    rgb.append(decimal)
  
  return tuple(rgb)

hex_to_rgb('FFA502')[1]

165

In [54]:
def random_hex_color(): 
	r = lambda: random.randint(0,255)
	return '#%02X%02X%02X' % (r(),r(),r())

randomColor = random_hex_color()
print(randomColor)
asRgb = hex_to_rgb(randomColor)
print(asRgb)

#F5AEBB
(245, 174, 187)


# Generate some training data

Create random colors, convert them between types, and output everything into a CSV file.

In [55]:
num_colors = 20000
colors = [random_hex_color() for _ in range(num_colors)]
rgb_values = [hex_to_rgb(color) for color in colors]

data = {'Hex': colors, 'R': [rgb[0] for rgb in rgb_values], 'G': [rgb[1] for rgb in rgb_values], 'B': [rgb[2] for rgb in rgb_values]}
df = pd.DataFrame(data)

df = df.drop_duplicates()
# Write the DataFrame to a CSV file
df.to_csv('rgb_colors.csv', index=False)

## Alternative data generation

Shamelessly stolen from chatgpt:

In [56]:
def rgb_to_hex(rgb):
    return ''.join([format(val, '02X') for val in rgb])

def hex_to_rgb(hex_str):
    return tuple(int(hex_str[i:i+2], 16) for i in range(0, 6, 2))

def generate_data(num_samples=1000):
    data = []
    for _ in range(num_samples):
        rgb = [random.randint(0, 255) for _ in range(3)]
        hex_str = rgb_to_hex(rgb)
        data.append((rgb, hex_str))
    return data

# Here we go...

In [57]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random


In [58]:
# Create the network architecture
class ColorPredictor(nn.Module):
    def __init__(self):
        super(ColorPredictor, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(3, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
        )
        self.output_layers = nn.ModuleList([nn.Linear(128, 4) for _ in range(6)])

    def forward(self, x):
        x = self.fc(x)
        return [output_layer(x) for output_layer in self.output_layers]

## Define the Training Function

In [59]:
def train(model, data, num_epochs=100, learning_rate=1e-3):
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        total_loss = 0.0
        random.shuffle(data)

        for rgb, hex_str in data:
            input_tensor = torch.tensor(rgb, dtype=torch.float32) / 255.0
            target_tensors = [torch.tensor(list(map(int, format(int(char, 16), '04b'))), dtype=torch.float32) for char in hex_str]

            optimizer.zero_grad()

            outputs = model(input_tensor)
            loss = sum([criterion(output, target_tensor) for output, target_tensor in zip(outputs, target_tensors)])
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(data)}")


In [60]:
model = ColorPredictor()
data = generate_data()
train(model, data)
torch.save(model.state_dict(), 'color_predictor.pth')

Epoch 1/100, Loss: 3.8403032214641573
Epoch 2/100, Loss: 3.6680932943820954
Epoch 3/100, Loss: 3.6342421395778657
Epoch 4/100, Loss: 3.5949534103870393
Epoch 5/100, Loss: 3.5341393692493437
Epoch 6/100, Loss: 3.473819903612137
Epoch 7/100, Loss: 3.410453906536102
Epoch 8/100, Loss: 3.3516240005493163
Epoch 9/100, Loss: 3.3148777453899383
Epoch 10/100, Loss: 3.284304325580597
Epoch 11/100, Loss: 3.2571546547412873
Epoch 12/100, Loss: 3.2400084664821627
Epoch 13/100, Loss: 3.2231007726192473
Epoch 14/100, Loss: 3.210009129047394
Epoch 15/100, Loss: 3.1924591579437256
Epoch 16/100, Loss: 3.1767403061389925
Epoch 17/100, Loss: 3.1680440282821656
Epoch 18/100, Loss: 3.157997973918915
Epoch 19/100, Loss: 3.1466429386138914
Epoch 20/100, Loss: 3.139504885196686
Epoch 21/100, Loss: 3.1244098286628725
Epoch 22/100, Loss: 3.120645533323288
Epoch 23/100, Loss: 3.1102321968078614
Epoch 24/100, Loss: 3.0990334696769715
Epoch 25/100, Loss: 3.100455060958862
Epoch 26/100, Loss: 3.0863240424394607
Epo